In [1]:
# 패키지
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 데이터 불러오기
path = "~/Dacon/서울시 자치구별 도보 네트워크 공간정보.csv"
df = pd.read_csv(path, encoding="cp949")
df['geometry'] = df['링크 WKT'].copy()
df.drop(columns=['링크 WKT', "노드링크 유형", "노드 WKT", "노드 ID",	"노드 유형 코드"], inplace=True)

print("Dimension of Data: ",df.shape)

# 공간정보 열에 NA가 있는 행 삭제
df = df.dropna(subset=['geometry'])

# 전체 도로 중 보행 가능만 필터
df['링크 유형 코드'] = df['링크 유형 코드'].astype(str).str[0:4]

txt = """1000
1001
1010
1011
1100
1101
1110
1111
0001
0010
0011"""
walker_LINK_CODE = txt.split('\n')
df = df[df["링크 유형 코드"].isin(walker_LINK_CODE)]

print("보행 가능 필터 후: ",df.shape)

# 공간정보 데이터로 바꾸기 gdf
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="epsg:4326")

Dimension of Data:  (491082, 18)
보행 가능 필터 후:  (278763, 18)


In [2]:
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString

def remove_dead_ends(roads_gdf, min_connection=2):
    """
    막다른 골목을 제거하는 함수
    
    Parameters:
    roads_gdf: 도로 네트워크가 포함된 GeoDataFrame
    min_connection: 최소 연결 수 (기본값: 2, 즉 양방향 연결)
    
    Returns:
    GeoDataFrame: 막다른 골목이 제거된 도로 네트워크
    """
    # 네트워크 그래프 생성
    G = nx.Graph()
    
    # 노드 딕셔너리 생성 (좌표를 키로 사용)
    node_connections = {}
    
    # 모든 도로의 시작점과 끝점을 노드로 추가
    for idx, row in roads_gdf.iterrows():
        coords = list(row.geometry.coords)
        for coord in coords:
            # 소수점 6자리까지만 사용하여 부동소수점 오차 방지
            coord = tuple(round(x, 6) for x in coord)
            if coord not in node_connections:
                node_connections[coord] = set()
    
        # 연속된 좌표들을 엣지로 추가
        for i in range(len(coords)-1):
            coord1 = tuple(round(x, 6) for x in coords[i])
            coord2 = tuple(round(x, 6) for x in coords[i+1])
            node_connections[coord1].add(coord2)
            node_connections[coord2].add(coord1)
            G.add_edge(coord1, coord2)
    
    # 충분한 연결이 없는 노드 식별
    dead_end_nodes = {node for node, connections in node_connections.items() 
                     if len(connections) < min_connection}
    
    # 막다른 골목이 포함된 도로 식별
    roads_to_keep = []
    for idx, row in roads_gdf.iterrows():
        coords = list(row.geometry.coords)
        coords = [tuple(round(x, 6) for x in coord) for coord in coords]
        
        # 도로의 시작점과 끝점이 모두 막다른 골목이 아닌 경우만 유지
        if not (coords[0] in dead_end_nodes or coords[-1] in dead_end_nodes):
            roads_to_keep.append(idx)
    
    # 결과 생성
    filtered_roads = roads_gdf.loc[roads_to_keep].copy()
    
    # 연결 정보 추가
    filtered_roads['num_connections'] = filtered_roads.apply(
        lambda row: min(
            len(node_connections[tuple(round(x, 6) for x in row.geometry.coords[0])]),
            len(node_connections[tuple(round(x, 6) for x in row.geometry.coords[-1])])
        ),
        axis=1
    )
    
    return filtered_roads

# 사용 예시
# 막다른 골목 제거
connected_roads = remove_dead_ends(gdf, min_connection=2)

In [5]:
connected_roads.to_csv("connected_road.csv", index=False)

In [8]:
# 패키지
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 데이터 불러오기
path = "/home/sungil/Byte-King-rawdata/connected_road.csv"
df = pd.read_csv(path)

In [19]:
df

,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,링크 길이,시군구코드,시군구명,읍면동코드,읍면동명,고가도로,지하철네트워크,교량,터널,육교,횡단보도,"공원,녹지",건물내,geometry,num_connections
0,67328,1111,160002.0,160467.0,9.772,1111000000,종로구,1111017400,창신동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,LINESTRING (127.01227965806434 37.572482022620...,3
1,62444,1111,152121.0,152120.0,40.736,1111000000,종로구,1111016700,충신동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,LINESTRING (127.00764209916748 37.574753776919...,3
2,65142,1111,148448.0,167354.0,91.654,1111000000,종로구,1111017400,창신동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,LINESTRING (127.00955647784441 37.579724508813...,3
3,75705,1111,190735.0,138465.0,19.612,1111000000,종로구,1111016800,동숭동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,LINESTRING (127.00586167477364 37.581208710843...,3
4,75491,1111,129219.0,146159.0,95.663,1111000000,종로구,1111018400,부암동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,LINESTRING (126.96709264424995 37.596115090220...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234324,1650,1000,159417.0,137693.0,25.227,1174000000,강동구,1174010600,둔촌동,0.0,0.0,0.0,0.0,0,0,0.0,1.0,LINESTRING (127.15215291470706 37.537834813896...,3
234325,92919,1111,155442.0,124154.0,42.069,1174000000,강동구,1174010900,천호동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,LINESTRING (127.12959029332069 37.539780536064...,3
234326,92507,1111,155242.0,155243.0,7.501,1174000000,강동구,1174010900,천호동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,LINESTRING (127.13236000969403 37.540129899039...,3
234327,67001,1111,58123.0,123665.0,24.741,1174000000,강동구,1174010900,천호동,0.0,0.0,0.0,0.0,0,0,0.0,0.0,"LINESTRING (127.1286726381813 37.543062337499,...",3


In [6]:
import folium
from folium import PolyLine
from tqdm import tqdm

M = folium.Map(location=[37.484208,126.929676], zoom_start=17, tiles='cartodbpositron')

# GeoDataFrame의 각 LINESTRING을 지도에 추가
for idx, row in tqdm(connected_roads.iterrows()):
    # LINESTRING 좌표 추출
    coordinates = [(y, x) for x, y in row.geometry.coords]

    # LineString 추가
    folium.PolyLine(
        locations=coordinates,
        weight=2,
        color='red',
        opacity=1.0,
        popup=f"LineString ID: {idx}"  # 팝업 정보 (필요에 따라 수정)
    ).add_to(M)

M.save("/home/sungil/Byte-King-rawdata/connected_road.html")

0it [00:00, ?it/s]

234329it [00:27, 8376.51it/s]
